# Project 1 : Collecting Data from Twitter

Start Date: 3/21/2022, Due Date: 3/31/2022, **BEFORE the beginning of class**

## **NOTE: There are *always* last minute issues submitting the case studies.  DO NOT WAIT UNTIL THE LAST MINUTE!**

<img src="https://logos-download.com/wp-content/uploads/2016/02/Twitter_Logo_new-700x569.png">

**Individual Assignment:** Please EDIT this cell and add your name

    Aidan Horn


**Suggested Readings:** 
* Chapter 1 and Chapter 9 of the book "Mining the Social Web" can help a lot if you get stuck. 
* In fact, it is intentional that many of these questions can be answered directly from there (except for question 4)!
* The idea is to ease you into the case studies :-)

**Don't forget!**
* You will need to install the twitter library to access the Twitter API
 * pip install twitter
* NOTE: There is a package called "python-twitter" which will not work with this notebook!


**NOTE**
* **Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost**.

# Problem 1 (20 points): Sampling Twitter Data with the Search API about a certain topic

* Select a topic that you are interested in, for example, "WPI" or "Lady Gaga"
* Use Twitter Search API to sample a collection of tweets about this topic. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million.
* Store the tweets you downloaded into a local file (txt file or json file) 

In [43]:
! pip install twitter

In [44]:
import twitter
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import json
from time import gmtime, strftime
#---------------------------------------------
# Define a Function to Login Twitter API
def oauth_login():
    # Prof. Paffenroth has a developer account for the class.  He will provide the Twitter access tokens for
    # each team
    # See https://developer.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'oJpnhPtow417C4xHxnzZ8icfS'
    CONSUMER_SECRET = 'i7eVuKl5LFQ1jhyRX3xyjH6HYgwEZtq0Av6YOBtC965PJ8cR9g'
    OAUTH_TOKEN = '1042905448528064512-z4LzLfbtR0CvPNoEmiQe1MfRD16aOB'
    OAUTH_TOKEN_SECRET = 'eb7MCzjwxH7zqxLsDbIJLapXS5j1tp79UkGPYH4h1FMu0'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

twitter_api = oauth_login()

In [45]:
print (strftime("%Y-%m-%d %H:%M:%S", gmtime()))

q = 'POTUS'
count = 200
# See https://dev.twitter.com/docs/api/1.1/get/search/tweets
search_results = twitter_api.search.tweets(q=q, count=count, tweet_mode = 'extended')

statuses = search_results['statuses']
#print([d['text'] for d in search_results['statuses']])

for _ in range(5):
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except (KeyError): # No more results when next_results doesn't exist
        break
    kwargs = dict([ kv.split('=') for kv in next_results[1:].split("&") ])
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

tweet = statuses[-1]['retweeted_status']['full_text'] if statuses[-1]['retweeted'] else statuses[-1]['text']
print("Latest tweet " + tweet)

2022-04-01 03:17:25
Length of statuses 86
Length of statuses 178
Length of statuses 273
Length of statuses 356
Length of statuses 445
Latest tweet @ClayTravis @clayandbuck @TuckerCarlson @CNN @FoxNews @MSNBC @DNC @RNC @HillaryClinton @RealCandaceO… https://t.co/JZ3IR9ZVSF


In [46]:
#File in local directory
filename = "statuses_POTUS.json"

with open(filename, 'r') as file:
    #Get list of json formatted tweet data
    list_s = json.load(file)
    
    print(f"Old length of tweet list: {len(list_s)}")
    #Extend list with list of statuses returned by twitter API search call in cell above
    list_s.extend(statuses)
    
    print(f"New length of tweet list: {len(list_s)}")
    
    #Use tweet id's to detect any duplicates in the list of statuses
    #Necessary due to occasional failure to detect duplicate tweets using 'not in unique_s'
    IDs = [sub['id'] for sub in list_s]
    duplicates = [ID for ID in IDs if IDs.count(ID) > 1]
    unique_duplicates = list(set(duplicates))
    
    #Create new list to append all non-duplicate entries
    unique_s = []

    for s in list_s:
        if s not in unique_s and s['id'] not in unique_duplicates:
            unique_s.append(s)
    
    print(f"Length of tweet list after removing duplicates: {len(unique_s)}")
    
with open(filename, 'w') as file:
    
    #Write list of unique statuses to file in json format
    json.dump(unique_s, file, indent = 1)

print("Latest tweet " + unique_s[-1]['text'])

Old length of tweet list: 4791
New length of tweet list: 5326
Length of tweet list after removing duplicates: 5326
Latest tweet @ClayTravis @clayandbuck @TuckerCarlson @CNN @FoxNews @MSNBC @DNC @RNC @HillaryClinton @RealCandaceO… https://t.co/JZ3IR9ZVSF


### Report some statistics about the tweets you collected 

* The topic of interest: POTUS 


* The total number of tweets collected:  4000+

* -----------------------

# Problem 2 (20 points): Analyzing Tweets and Tweet Entities with Frequency Analysis

**1. Word Count:** 
* Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets. 
* Plot a table of the top 30 words with their counts

In [47]:
filename = "statuses_POTUS.json"

with open(filename, 'r') as file:
    list_s = json.load(file)  

list_t = []

#Some tweets in list are were added before tweet_mode was extended in the API search call
#The content of these tweets are indexed at either "text" or "full_text"

for s in list_s:
    try:
        t = s['text']
    except(KeyError):
        t = s['full_text']
    list_t.append(t)

In [48]:

#most of the common words will be artifacts such as https, punctuation, or very common words
#we will use a stopword set from nltk modified to include other unwanted words to filter the tweet text
#we will also use a word tokenizer to filter out punctuation marks
#finally, we will ensure all text is lowercase

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['https','co','amp','rt','potus'])
filtered_words = [w.lower() for t in list_t 
                  for w in tokenizer.tokenize(t) 
                  if not w.lower() in stop_words]

c = Counter(filtered_words)
#print(c.most_common()[:30])

df_count = pd.DataFrame(data = c.most_common()[:30], columns=["Word", "Count"], index=list(range(1,31)))
df_count.to_csv('df_count.csv')
df_count.head(30)

,Word,Count
1,tigray,405
2,vp,346
3,000,314
4,today,286
5,us,285
6,people,281
7,till,266
8,emmett,264
9,act,254
10,know,242


In [49]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = " ".join(word for word in df_count.Word)
wordcloud = WordCloud(background_color="white").generate(text)
wordcloud.to_file("top_30_words.png")

plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

<Figure size 640x480 with 1 Axes>

**2. Find the most popular tweets in your collection of tweets**

Please plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.


In [50]:
filename = "statuses_POTUS.json"

with open(filename, 'r') as file:
    list_s = json.load(file)

rt = []

for s in list_s:
    try:
        if 'retweeted_status' in s:
            rt.append((s['retweet_count'], s['retweeted_status']['user']['screen_name'], s['text']))
        
    except(KeyError):
        if 'retweeted_status' in s:        
            rt.append((s['retweet_count'], s['retweeted_status']['user']['screen_name'], s['full_text']))
            
df_rt = pd.DataFrame(data = sorted(rt, reverse = True), columns=["Count", "Screen Name", "Text"])
df_rt = df_rt.drop_duplicates(subset = ['Text'])
df_rt.to_csv('df_rt.csv')
df_rt.head(10)

,Count,Screen Name,Text
0,24536,POTUS,RT @POTUS: I want to be clear: We will defend ...
1,21024,POTUS,RT @POTUS: Putin has the gall to say he is “de...
6,20417,POTUS,RT @POTUS: After my predecessor’s fiscal misma...
50,19181,POTUS,RT @POTUS: I visited Ukrainian refugees who ha...
55,17954,ZelenskyyUa,RT @ZelenskyyUa: Had a substantive conversatio...
56,17727,AVindman,RT @AVindman: This was an historical speech fo...
58,14316,POTUS,RT @POTUS: I just signed the Emmett Till Antil...
91,13635,POTUS,"RT @POTUS: This afternoon, I met with members ..."
92,11611,senadormx,RT @senadormx: Yo le propongo a @KenSalazar d...
93,11228,POTUS,RT @POTUS: We are engaged anew in a great batt...


**3. Find the most popular Tweet Entities in your collection of tweets**

Please plot a table of the top 10 hashtags, top 10 user mentions that are the most popular in your collection of tweets.

In [51]:
filename = "statuses_POTUS.json"

with open(filename, 'r') as file:
    list_s = json.load(file)

ht = [hashtag['text'] for s in list_s for hashtag in s['entities']['hashtags']]

c = Counter(ht)

#um = [user_mention['name'] for s in list_s for user_mention in s['entities']['hashtags']]

df_ht = pd.DataFrame(data = c.most_common()[:10], columns = ['Hashtag','Count'], index = list(range(1,11)))
df_ht.to_csv('df_ht.csv')
df_ht.head(10)

,Hashtag,Count
1,Tigray,200
2,AllowAccessToTigray,64
3,SupportHR6600,59
4,IC,58
5,AntilynchingAct,57
6,CancelStudentDebt,48
7,Ethiopian,35
8,TigrayGenocide,33
9,Ethiopia,31
10,strisciaforpeace,30


In [52]:
filename = "statuses_POTUS.json"

with open(filename, 'r') as file:
    list_s = json.load(file)

um = [user_mention['name'] for s in list_s for user_mention in s['entities']['user_mentions']]

c = Counter(um)

df_um = pd.DataFrame(data = c.most_common()[:10], columns = ['User Mention','Count'], index = list(range(1,11)))
df_um.to_csv('df_um.csv')
df_um.head(10)

,User Mention,Count
1,President Biden,3575
2,Vice President Kamala Harris,336
3,Boris Johnson,150
4,Secretary Antony Blinken,109
5,Joe Biden,103
6,Emmanuel Macron,100
7,Olaf Scholz,91
8,USAID,87
9,NATO,85
10,Bobby L. Rush,79


* ------------------------

# Problem 3 (20 points): Getting "All" friends and "All" followers of a popular user in twitter


* choose a popular twitter user who has many followers. Since some popular users have millions of followers, it may take a very long time to get the follower list. Hence, we recommend you choosing a "popular" user with a reasonable number of followers, that your program can handle.
* Get the list of all friends and all followers of the twitter user.
* Plot 20 out of the followers, plot their ID numbers and screen names in a table.
* Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.

In [53]:
import tweepy
#---------------------------------------------
# Define a Function to Login Twitter API
def tweepy_oauth_login():
    # Prof. Paffenroth has a developer account for the class.  He will provide the Twitter access tokens for
    # each team
    # See https://developer.twitter.com/docs/auth/oauth for more information 
    # on Twitter's OAuth implementation.
    
    CONSUMER_KEY = 'oJpnhPtow417C4xHxnzZ8icfS'
    CONSUMER_SECRET = 'i7eVuKl5LFQ1jhyRX3xyjH6HYgwEZtq0Av6YOBtC965PJ8cR9g'
    OAUTH_TOKEN = '1042905448528064512-z4LzLfbtR0CvPNoEmiQe1MfRD16aOB'
    OAUTH_TOKEN_SECRET = 'eb7MCzjwxH7zqxLsDbIJLapXS5j1tp79UkGPYH4h1FMu0'
    
    auth = tweepy.OAuth1UserHandler(
       CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
    
    return tweepy.API(auth, wait_on_rate_limit=True)

api = tweepy_oauth_login()


In [54]:
import time
import pandas as pd

screen_name = "BobbyBroccole"

#followers = api.get_followers(screen_name = screen_name, count = 200)

users = tweepy.Cursor(api.get_followers, screen_name=screen_name, count = 200).items(1206)
u = []
while True:
    try:
        user = next(users)
    except tweepy.TweepyException:
        time.sleep(60*15)
        user = next(users)
    except StopIteration:
        break
    #print(user.screen_name)
    u.append((user.id,user.screen_name))

df_fl = pd.DataFrame(data = u, columns = ["ID","Screen Name"])

In [55]:
df_fl.head(30)

,ID,Screen Name
0,757634771853934592,Nynebark
1,1077703096317415430,braillecollage
2,1330157180994064387,tolzasso
3,1329663294567813121,ThatNaughtyKat
4,948653033407369216,unniFI
5,1350199414544072709,bojojojo02
6,1360385187251539971,IamAlexChao
7,3965371,eduardoe
8,1399259474079014917,SteelwoolOcto
9,1362214222877192194,3892534627a


In [56]:
import time

screen_name = "BobbyBroccole"

users = tweepy.Cursor(api.get_friends, screen_name=screen_name, count = 200).items(363)
u = []
while True:
    try:
        user = next(users)
    except tweepy.TweepyException:
        time.sleep(60*15)
        user = next(users)
    except StopIteration:
        break
    #print(user.screen_name)
    u.append((user.id,user.screen_name))

df_fr = pd.DataFrame(data = u, columns = ["ID","Screen Name"])

In [57]:
df_fr.head(20)

,ID,Screen Name
0,1506689287278014469,PoliticDrip
1,1411723858181251074,ampol_moment
2,1504956528914309122,ClubPhotos_
3,741809888368566272,GreatCheshire
4,840848485,FredInTheKnud
5,36201559,VorosTwins
6,984862457217933318,KevinPerjurer
7,318993598,Q_Review
8,2616274436,CJMacaulay
9,993692561863667712,OOHKAYEYE


* Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table

In [58]:
intersection = pd.merge(df_fl,df_fr,how='inner')

intersection.head()

,ID,Screen Name
0,318993598,Q_Review
1,70739029,jon_bois
2,993692561863667712,OOHKAYEYE
3,29472053,Hbomberguy
4,2330900772,GC__Vazquez


* ------------------------

# Problem 4 (20 points): Business question 

Run some additional experiments with your data to gain familiarity with the twitter data and twitter API.

* Come up with a business question that Twitter data could help answer.
* Decribe the business case.
* How could Twitter data help a company decide how to spend its resources.

### Business Question:
#### *In order to improve a brand's Twitter presence, how does one prioritize which Tweets at the company to which a social media manager should respond directly?*

### Business Case:

* Let's say a business, such as Nestle, feels they lack a social media presence on Twitter. 

* In order to establish a notable awareness among users and a good reputation, they believe they should increase their user engagement by responding to Tweets at their account. 

* However, they want to find a way to prioritize content that is emotionally charged, either strongly positive or strongly negative, as well as increase engagement by replying to Tweets that will increase in popularity. 
* Therefore, they need a system that can aggregate recent tweets and rank Tweets by the predicted impact, which can be estimated from the intensity of the language and the number of retweets and favorites.

#### How could Twitter data help a company decide how to spend its resources?

* Twitter is a useful tool for extracting public sentiment
* Marketing departments spend millions to determine how the public feel about a company and how to improve a brand's image.
* However, small scale interactions with consumers or members of the public are often more effective than large scale advertisments and PR campaigns, but these efforts require a high level of engagement thus a high amount of resources necessary.
* We can more efficiently allocate these resources if we can specifically target individuals who are already engaging with the brand and who may act as a nexus for others to engage.
* Thus Twitter data can help identify these individuals and improve the decision making abilities of a company.

> Start by pulling recent tweets directed at the brand's twitter handle

In [59]:
q = "@Nestle"

search_results = twitter_api.search.tweets(q=q, count=200, tweet_mode = 'extended', lang='en')

statuses = search_results['statuses']

> Create a database of relevant information about the tweet

> Filter out Retweets to avoid duplicate text bodies

In [60]:
reply_data = [(r['id_str'],
                r['user']['name'],
                r['full_text'],
                r['retweet_count'],
                r['favorite_count']) for r in statuses if r['full_text'][0:2] != 'RT']

df_replies = pd.DataFrame(data = reply_data, 
                          columns = ['ID','Screen Name','Tweet','Retweet Count','Favorited Count'])

df_replies.to_csv('df_replies.csv')
df_replies.sort_values(by = 'Retweet Count', ascending = False).head()

,ID,Screen Name,Tweet,Retweet Count,Favorited Count
22,1509632999020642315,Tenacious “T” #VoteBlueToEndTheViolence 🌻,Stop @nestle @ArrowheadWater from taking milli...,1,3
23,1509630993455583236,ПАПАndopulo,"Hey, @Nestle @Nestle_es @NestleColombia @Nestl...",1,0
0,1509729253482450944,Dr. Magdalena Whoolery,"Beware @Nestle’s “help”4Ukraine. “Nestlé, the ...",0,1
26,1509629920561012739,k,@Nestle @IrishTimes Russia is a murderer and a...,0,0
20,1509635996572270611,RafmansKitchen,Out of coffee and creamer? No biggie we got y...,0,0


> Now we can run sentiment intensity analysis on the tweet content, we just want the compound score

In [61]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()
df_replies['Compound SA Score'] = [sid.polarity_scores(t)['compound'] for t in df_replies['Tweet']]
df_replies.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\horna\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,ID,Screen Name,Tweet,Retweet Count,Favorited Count,Compound SA Score
0,1509729253482450944,Dr. Magdalena Whoolery,"Beware @Nestle’s “help”4Ukraine. “Nestlé, the ...",0,1,-0.7251
1,1509724594919288833,Wear A Mask and Please take the vaccine!,@AnonOpsSE @Nestle @Gizmodo Dear Anonymous \n@...,0,0,0.3818
2,1509719066046603273,Dr. Magdalena Whoolery,@AnonOpsSE @Nestle @Gizmodo Trust me. Nestlé a...,0,1,0.6597
3,1509707111529332741,ꀘꃅ4ꋪꂵꍏ ꀘ0ꂵꍏ,@AnonOpsSE @Nestle @Gizmodo your parents didn'...,0,0,0.3096
4,1509706896810389508,FD Svensson 🍀 #FCKPTN,@AnonOpsSE @Nestle @Gizmodo It's not just abou...,0,0,-0.9042


In [62]:
sid.polarity_scores(df_replies['Tweet'][10])

{'neg': 0.19, 'neu': 0.81, 'pos': 0.0, 'compound': -0.6608}

> We want to estimate the amount of impact a tweet may have by creating a score using retweet count, favorited count, and the SA score

> We wnat to priortize retweets over favorites since they will engage with more users

In [63]:
df_replies['Impact Score'] = (df_replies['Retweet Count'] + 1
                              + df_replies['Favorited Count']/4) * df_replies['Compound SA Score']
df_replies.head()

,ID,Screen Name,Tweet,Retweet Count,Favorited Count,Compound SA Score,Impact Score
0,1509729253482450944,Dr. Magdalena Whoolery,"Beware @Nestle’s “help”4Ukraine. “Nestlé, the ...",0,1,-0.7251,-0.906375
1,1509724594919288833,Wear A Mask and Please take the vaccine!,@AnonOpsSE @Nestle @Gizmodo Dear Anonymous \n@...,0,0,0.3818,0.381800
2,1509719066046603273,Dr. Magdalena Whoolery,@AnonOpsSE @Nestle @Gizmodo Trust me. Nestlé a...,0,1,0.6597,0.824625
3,1509707111529332741,ꀘꃅ4ꋪꂵꍏ ꀘ0ꂵꍏ,@AnonOpsSE @Nestle @Gizmodo your parents didn'...,0,0,0.3096,0.309600
4,1509706896810389508,FD Svensson 🍀 #FCKPTN,@AnonOpsSE @Nestle @Gizmodo It's not just abou...,0,0,-0.9042,-0.904200


> Finally we want to sort this database by the magnitude of the impact score. Positive tweets and negative tweets have an equal weight in this case.

In [64]:
df_priority = df_replies.iloc[(-df_replies['Impact Score'].abs()).argsort()]
df_priority.head()

,ID,Screen Name,Tweet,Retweet Count,Favorited Count,Compound SA Score,Impact Score
24,1509630079374082049,Anonymous Operations,It's lovely how @Nestle completely ignore the ...,0,3,-0.6066,-1.061550
7,1509685084252786690,Mike's One of Millions,@aaronhoyland As long as you steer clear of @N...,0,3,0.5859,1.025325
6,1509692534393643013,Suzie Gray,Let’s BOYCOTT @Nestle ! Nestle is a Murderer ...,0,0,-0.9103,-0.910300
0,1509729253482450944,Dr. Magdalena Whoolery,"Beware @Nestle’s “help”4Ukraine. “Nestlé, the ...",0,1,-0.7251,-0.906375
4,1509706896810389508,FD Svensson 🍀 #FCKPTN,@AnonOpsSE @Nestle @Gizmodo It's not just abou...,0,0,-0.9042,-0.904200


> This experiment has produced a worthwhile tweet to respond to. A legitmate customer concern with a larger number of retweets and favorites, indicating public sentiment is aligned with this persons opinion.

* -----------------
# Done

All set! 

**What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 10-15 minutes talk) to present about the project. We will ask two students which are randomly selected to present their results in class for this project. 

* **Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


**How to submit:**

        Please submit in canvas.

#### We auto-process the submissions so make sure your subject line is *exactly*:

# Grading Criteria:

**Totoal Points: 100**


---------------------------------------------------------------------------
**Notebook results:**
    Points: 80


    -----------------------------------
    Qestion 1:
    Points: 20
    -----------------------------------
    
    (1) Select a topic that you are interested in.
    Points: 6 
    
    (2) Use Twitter Streaming API to sample a collection of tweets about this topic in real time. (It would be recommended that the number of tweets should be larger than 200, but smaller than 1 million. Please check whether the total number of tweets collected is larger than 200?
    Points: 10 
    
    
    (3) Store the tweets you downloaded into a local file (txt file or json file)
    Points: 4 
    
    
    -----------------------------------
    Qestion 2:
    Points: 20
    -----------------------------------
    
    1. Word Count

    (1) Use the tweets you collected in Problem 1, and compute the frequencies of the words being used in these tweets.
    Points: 4 

    (2) Plot a table of the top 30 words with their counts 
    Points: 4 
    
    2. Find the most popular tweets in your collection of tweets
    plot a table of the top 10 tweets that are the most popular among your collection, i.e., the tweets with the largest number of retweet counts.
    Points: 4 
    
    3. Find the most popular Tweet Entities in your collection of tweets

    (1) plot a table of the top 10 hashtags, 
    Points: 4 

    (2) top 10 user mentions that are the most popular in your collection of tweets.
    Points: 4 
    
    
    -----------------------------------
    Qestion 3:
    Points: 20
    -----------------------------------
    
    (1) choose a popular twitter user who has many followers, such as "ladygaga".
    Points: 4 

    (2) Get the list of all friends and all followers of the twitter user.
    Points: 4 

    (3) Plot 20 out of the followers, plot their ID numbers and screen names in a table.
    Points: 4 

    (4) Plot 20 out of the friends (if the user has more than 20 friends), plot their ID numbers and screen names in a table.
    Points: 4 
    
    (5) Compute the mutual friends within the two groups, i.e., the users who are in both friend list and follower list, plot their ID numbers and screen names in a table
    Points: 4 
  
    -----------------------------------
    Qestion 4:  Business question
    Points: 20
    -----------------------------------
        Novelty: 10
        Interestingness: 10
    -----------------------------------
    Run some additional experiments with your data to gain familiarity with the twitter data ant twitter API.  Come up with a business question and describe how Twitter data can help you answer that question.





---------------------------------------------------------------------------
**Slides (for presentation): Story-telling**
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 

